# Workplace to test various codes

In [1]:
import numpy as np
import os

import netCDF4
from netCDF4 import Dataset
from netCDF4 import date2num, num2date

import calendar
import datetime as dt

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')

from read_AM_index_mod import Reanalysis

In [4]:
#=============================================================
# class CMIP()
# Read the annular modes from cmip data
#=============================================================
class CMIP():
    def __init__(self, name_dir, year_start, year_end, name, source_dir=None, \
                 annual_cycle_fft=4, running_mean=0, save_AM_index=False):
        """
        Calculate the annular mode indices from pre-processed geopotential height averaged over the polar caps. \\
        The input data is daily. The global mean height is removed. The AM indices are normalized. \\
        The indices are saved as 
            NAM_index(year*days_in_a_year, pressure),
            SAM_index(year*days_in_a_year, pressure)

        Key options
        annual_cycle_fft: option to remove the harmonics above `annual_cycle_fft` from the annual cycle
        running_mean: option to conduct running average for the anomaly with `running_mean`

        References
        Gerber, E. P., and P. Martineau, 2018: Quantifying the variability of the annular modes: 
        Reanalysis uncertainty vs. sampling uncertainty. Atmos. Chem. Phys., 18, 17099–17117, https://doi.org/10.5194/acp-18-17099-2018.
        """

        self.name_dir = name_dir
        self.year_start = year_start
        self.year_end = year_end
        self.num_years = year_end - year_start + 1
        self.name = name
                
        self.root_dir = os.getcwd()
        if source_dir is None:
            self.source_dir = self.root_dir    # use cwd as source_dir
        else:
            self.source_dir = source_dir    # specify source_dir
        
        file = self.source_dir + '/' + self.name_dir + '/AM_daily_' \
            + str(self.year_start) + '_' + str(self.year_end) + '.nc'
        ncfile = Dataset(file, 'r')
        self.level = ncfile.variables['plev'][:]
        if ncfile.variables['plev'].units == 'Pa':      # convert Pa to hPa
            self.level /= 100.
        self.num_levels = len(self.level)
        
        self.annual_cycle_fft = annual_cycle_fft
        self.running_mean = running_mean
        
        if annual_cycle_fft != 4 or running_mean != 0:      # default values
            tag = f"_fft{annual_cycle_fft}_rm{running_mean}"
        else:
            tag = ""
        self.file_save = self.root_dir + '/' + self.name_dir + '/AM_daily_' \
            + str(self.year_start) + '_' + str(self.year_end) + tag + '.nc'
        
        if os.path.exists(self.file_save):
            print('Reading from saved data ......')
            self.NAM, self.SAM = self.read_AM_index()
        else:
            print('Calculating from the original data .......')
            self.NAM, self.SAM = self.cal_AM_index()
            if save_AM_index:
                self.save_AM_index()


    def read_AM_index(self):
        """
        Reading the AM indices from 'file_save'
        """
        
        ncfile = Dataset(self.file_save, mode='r') 

        return ncfile.variables['NAM'][:], ncfile.variables['SAM'][:]

    def get_data(self, var_name):
        """
        read data from year_start to year_end
        return var_o(year*days_in_a_year, pressure),    # total field
               var_mean_o(days_in_a_year, pressure),    # annual cycle, option to remove
                                                        # the harmonics above `annual_cycle_fft`
               var_anomaly_o(year*days_in_a_year, pressure) # anomaly, option to conduct
                                                            # running average with `running_mean`
        """

        file = self.source_dir + '/' + self.name_dir + '/AM_daily_' + str(self.year_start) + '_' + str(self.year_end) + '.nc'
        ncfile = Dataset(file, 'r')
        var_o = ncfile.variables[var_name][:]        # dim(year*days_in_a_year, pressure)
        var = var_o.reshape(-1, 365, self.num_levels)   # dim(year, days_in_a_year, pressure)
        # print(f'Length of data: {var_o.shape[0]}')
        
        var_mean = var.mean(axis=0)
        var_mean_o = np.empty_like(var_mean)
        if self.annual_cycle_fft > 0:
            var_mean_fft = np.fft.fft(var_mean, axis=0)
            var_mean_fft[self.annual_cycle_fft+1:-self.annual_cycle_fft, :] = 0
            var_mean_o = (np.fft.ifft(var_mean_fft, axis=0)).real
        else:
            var_mean_o = var_mean

        var_anomaly = (var - var_mean_o).reshape(-1, self.num_levels)    # broadcasting the 1st dimension        
        var_anomaly_o = np.empty_like(var_anomaly)
        if self.running_mean > 0:
            for k in range(self.num_levels):
                var_anomaly_o[:, k] = np.convolve(var_anomaly[:, k], 
                                               np.ones((self.running_mean,))/self.running_mean, mode='same')
        else:
            var_anomaly_o = var_anomaly

        return var_o, var_mean_o, var_anomaly_o

    def cal_AM_index(self):
        """
        Calculating the annular mode indices from the original data
        removing the global mean and standardize the anomaly time series

        return NAM_index(year*days_in_a_year, pressure),
               SAM_index(year*days_in_a_year, pressure)
        """

        NAM, NAM_mean, NAM_anomaly = self.get_data('NAM')
        SAM, SAM_mean, SAM_anomaly = self.get_data('SAM')
        GLOBAL, GLOBAL_mean, GLOBAL_anomaly = self.get_data('Global')

        NAM_index = -(NAM_anomaly-GLOBAL_anomaly)
        SAM_index = -(SAM_anomaly-GLOBAL_anomaly)

        NAM_index /= NAM_index.std(axis=0)
        SAM_index /= SAM_index.std(axis=0)

        return NAM_index, SAM_index

    def save_AM_index(self):
        """
        save the annular mode indices in 'file_save'

        see more about the netCDF4 package at
        https://unidata.github.io/python-training/workshop/Bonus/netcdf-writing/
        """
            
        # Opening a file
        # try: ncfile.close()  # just to be safe, make sure dataset is not already open.
        # except: pass

        if not os.path.exists(os.path.dirname(self.file_save)):
            os.makedirs(os.path.dirname(self.file_save))
            
        ncfile = Dataset(self.file_save, mode='w', format='NETCDF4_CLASSIC') 

        # Creating dimensions
        pressure_dim = ncfile.createDimension('pressure', self.num_levels)
        time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).

        # Creating attributes
        ncfile.title='Annular Mode indices'

        # Creating coordinate and data variables
        pressure = ncfile.createVariable('pressure', np.float32, ('pressure',))
        pressure.units = 'hPa'
        pressure.long_name = 'Pressure level'

        time = ncfile.createVariable('time', np.float64, ('time',))
        time.units = 'hours since 1800-01-01'
        time.long_name = 'time'
        time.calendar = 'noleap'

        NAM = ncfile.createVariable('NAM',np.float32,('time','pressure')) # note: unlimited dimension is leftmost
        NAM.units = ''
        NAM.standard_name = 'Northern Annular Mode Index'

        SAM = ncfile.createVariable('SAM',np.float32,('time','pressure')) # note: unlimited dimension is leftmost
        SAM.units = ''
        SAM.standard_name = 'Southern Annular Mode Index'

        # Writing data
        # Note: the ":" is necessary in these "write" statements
        pressure[:] = self.level

        date_start = dt.datetime(self.year_start, 1, 1, 0)
        date_end = dt.datetime(self.year_end, 12, 31, 0)
                    
        num_start = date2num(date_start, units=time.units, calendar=time.calendar)
        num_end = date2num(date_end, units=time.units, calendar=time.calendar)
        time[:] = np.linspace(num_start, num_end, 365*(self.year_end-self.year_start+1))
        #print(num2date(time[0:4], units=time.units, calendar=time.calendar))
        #print(num2date(time[-4:], units=time.units, calendar=time.calendar))

        NAM[:,:] = self.NAM
        SAM[:,:] = self.SAM
        
        # Closing a netCDF file
        print(ncfile)
        ncfile.close()


D = CMIP(name_dir='GFDL-ESM4', year_start=1950, year_end=2014, name='GFDL-ESM4', source_dir='cmip6', \
                        annual_cycle_fft=4, running_mean=0, save_AM_index=True)


SyntaxError: unexpected character after line continuation character (1633075217.py, line 51)

In [ ]:
print(D.level[:])
print(D.file_save)